# Basic usage of the ``empix`` library #

## A NOTE BEFORE STARTING ##

Since the ``empix`` git repository tracks this notebook under its original
basename ``basic_usage.ipynb``, we recommend that you copy the original notebook
and rename it to any other basename that is not one of the original basenames
that appear in the ``<root>/examples`` directory before executing any of the
notebook cells below, where ``<root>`` is the root of the ``empix``
repository. This way you can explore the notebook by executing and modifying
cells without changing the original notebook, which is being tracked by git.

## Import necessary modules ##

In [ ]:
# For general array handling and constructing random number generators.
import numpy as np

# For creating hyperspy signals.
import hyperspy.signals
import hyperspy.axes



# The library that is the subject of this demonstration.
import empix

In [ ]:
%matplotlib ipympl
%matplotlib ipympl

## Introduction ##

In this notebook, we demonstrate how one can use each function and class in the
``empix`` library.

You can find the documentation for the ``empix`` library
[here](https://mrfitzpa.github.io/empix/_autosummary/empix.html).  It is
recommended that you consult the documentation of this library as you explore
the notebook. Moreover, users should execute the cells in the order that they
appear, i.e. from top to bottom, as some cells reference variables that are set
in other cells above them.

## Using the ``empix`` library ##

### Creating the input ``hyperspy`` signals. ###

Let's create two 2D ``hyperspy`` signals to use as input signals for most of the
functions in the ``empix`` library.

In [ ]:
def generate_real_2d_input_signal():
    complex_2d_input_signal = generate_complex_2d_input_signal()
    
    real_2d_input_signal = complex_2d_input_signal.real
    real_2d_input_signal.metadata.General.title = "First Real 2D Input"
    real_2d_input_signal.metadata.max_imag_val_inside_ring = 0
    real_2d_input_signal.metadata.avg_imag_val_inside_ring = 0

    real_2d_input_signal.axes_manager[-2].name = "$x$"
    real_2d_input_signal.axes_manager[-2].units = "Å"

    real_2d_input_signal.axes_manager[-1].name = "$y$"
    real_2d_input_signal.axes_manager[-1].units = "Å"

    return real_2d_input_signal



def generate_complex_2d_input_signal():
    kwargs = {"data": generate_complex_2d_input_signal_data(), 
              "metadata": generate_complex_2d_input_signal_metadata()}
    complex_2d_input_signal = hyperspy.signals.ComplexSignal2D(**kwargs)

    axes = generate_complex_2d_input_signal_axes()

    for axis_idx, axis in enumerate(axes):
        complex_2d_input_signal.axes_manager[axis_idx].update_from(axis)
        complex_2d_input_signal.axes_manager[axis_idx].name = axis.name
        complex_2d_input_signal.axes_manager[axis_idx].units = axis.units

    return complex_2d_input_signal



def generate_complex_2d_input_signal_data():
    signal_data_shape = generate_complex_2d_input_signal_data_shape()
    Y_dim, X_dim, v_dim, h_dim = signal_data_shape

    metadata = generate_complex_2d_input_signal_metadata()
    ring_centers_in_pixel_coords = metadata["ring_centers_in_pixel_coords"]
    inner_ring_radius_in_pixels = metadata["inner_ring_radius_in_pixels"]
    outer_ring_radius_in_pixels = metadata["outer_ring_radius_in_pixels"]
    max_real_val_inside_ring = metadata["max_real_val_inside_ring"]
    max_imag_val_inside_ring = metadata["max_imag_val_inside_ring"]

    kwargs = {"shape": signal_data_shape, "dtype": "complex"}
    signal_data = np.zeros(**kwargs)

    navigation_dims = (Y_dim, X_dim)
    num_patterns = Y_dim*X_dim

    cols = np.arange(h_dim)
    rows = np.arange(v_dim)

    pair_of_1d_coord_arrays = (cols, rows)
    n_h, n_v = np.meshgrid(*pair_of_1d_coord_arrays, indexing="xy")

    for pattern_idx in range(num_patterns):
        navigation_indices = np.unravel_index(pattern_idx, navigation_dims)
        Y_idx, X_idx = navigation_indices
        
        n_h_c, n_v_c = ring_centers_in_pixel_coords[Y_idx][X_idx]

        n_hv = np.sqrt((n_h-n_h_c)**2 + (n_v-n_v_c)**2)

        signal_data.real[navigation_indices] = \
            (inner_ring_radius_in_pixels <= n_hv)
        signal_data.real[navigation_indices] *= \
            (n_hv <= outer_ring_radius_in_pixels)
        signal_data.imag[navigation_indices] = \
            signal_data.real[navigation_indices]
        signal_data.real[*navigation_indices, :, n_h_c:] *= \
            max_real_val_inside_ring
        signal_data.imag[*navigation_indices, :, n_h_c:] *= \
            max_imag_val_inside_ring

    complex_2d_input_signal_data = signal_data

    return complex_2d_input_signal_data



def generate_complex_2d_input_signal_data_shape():
    complex_2d_input_signal_data_shape = (3, 2, 181, 200)
    
    return complex_2d_input_signal_data_shape



def generate_complex_2d_input_signal_metadata():
    axes = generate_complex_2d_input_signal_axes()
    h_offset = axes[-2].offset
    h_scale = axes[-2].scale
    v_offset = axes[-1].offset
    v_scale = axes[-1].scale

    ring_centers_in_pixel_coords = generate_ring_centers_in_pixel_coords()
    
    ring_centers = np.array(ring_centers_in_pixel_coords, dtype="float")
    ring_centers[:, :, 0] = h_offset + ring_centers[:, :, 0]*h_scale
    ring_centers[:, :, 1] = v_offset + ring_centers[:, :, 1]*v_scale
    ring_centers = ring_centers.tolist()

    inner_ring_radius_in_pixels = 50
    outer_ring_radius_in_pixels = 60
    inner_ring_radius = abs(h_scale)*inner_ring_radius_in_pixels
    outer_ring_radius = abs(h_scale)*outer_ring_radius_in_pixels

    max_real_val_inside_ring = 2
    max_imag_val_inside_ring = 3

    avg_real_val_inside_ring = (1+max_real_val_inside_ring)/2
    avg_imag_val_inside_ring = (1+max_imag_val_inside_ring)/2
    
    metadata = {"General": {"title": "First Complex 2D Input"}, 
                "Signal": dict(), 
                "ring_centers_in_pixel_coords": ring_centers_in_pixel_coords, 
                "ring_centers": ring_centers,
                "inner_ring_radius_in_pixels": inner_ring_radius_in_pixels, 
                "outer_ring_radius_in_pixels": outer_ring_radius_in_pixels, 
                "inner_ring_radius": inner_ring_radius, 
                "outer_ring_radius": outer_ring_radius, 
                "max_real_val_inside_ring": max_real_val_inside_ring,
                "max_imag_val_inside_ring": max_imag_val_inside_ring,
                "avg_real_val_inside_ring": avg_real_val_inside_ring, 
                "avg_imag_val_inside_ring": avg_imag_val_inside_ring}

    complex_2d_input_signal_metadata = metadata

    return complex_2d_input_signal_metadata



def generate_ring_centers_in_pixel_coords():
    signal_data_shape = generate_complex_2d_input_signal_data_shape()
    Y_dim, X_dim, v_dim, h_dim = signal_data_shape
    
    ring_centers_in_pixel_coords = np.zeros((Y_dim, X_dim, 2), dtype="int")

    for Y_idx in range(Y_dim):
        for X_idx in range(X_dim):
            n_h_c = (h_dim-1)/2 + 10*(X_idx - (X_dim//2))
            n_v_c = (v_dim-1)/2 + 10*(Y_idx - (Y_dim//2))
            
            ring_centers_in_pixel_coords[Y_idx, X_idx, 0] = n_h_c
            ring_centers_in_pixel_coords[Y_idx, X_idx, 1] = n_v_c

    ring_centers_in_pixel_coords = ring_centers_in_pixel_coords.tolist()

    return ring_centers_in_pixel_coords



def generate_complex_2d_input_signal_axes():
    signal_data_shape = generate_complex_2d_input_signal_data_shape()
    Y_dim, X_dim, v_dim, h_dim = signal_data_shape

    d_h = 0.1
    d_v = -d_h

    axes_sizes = (X_dim, Y_dim, h_dim, v_dim)
    axes_scales = (1, 7, d_h, d_v)
    axes_offsets = (11, 22, -((h_dim-1)//2)*d_h, ((v_dim-1)//2)*abs(d_v))
    axes_names = ("$X$", "$Y$", "$k_x$", "$k_y$")
    axes_units = ("mm", "mm", "1/Å", "1/Å")

    axes = tuple()
    for axis_idx, _ in enumerate(axes_names):
        kwargs = {"size": axes_sizes[axis_idx],
                  "scale": axes_scales[axis_idx],
                  "offset": axes_offsets[axis_idx],
                  "name": axes_names[axis_idx], 
                  "units": axes_units[axis_idx]}
        axis = hyperspy.axes.UniformDataAxis(**kwargs)
        axes += (axis,)

    complex_2d_input_signal_axes = axes

    return complex_2d_input_signal_axes



real_2d_input_signal = generate_real_2d_input_signal()
complex_2d_input_signal = generate_complex_2d_input_signal()

Let's plot the 2D input signals that we just created. Starting with the 
real-valued signal.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
real_2d_input_signal.plot(**kwargs)

And now the complex-valued signal.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
complex_2d_input_signal.plot(**kwargs)

Note that we modified the signal axes from their default states. In particular,
we rescaled the signal space axes scales by a factor of 10. The default/original
signal space scales were equal to unity.

Let's also create two 2D ``hyperspy`` signals to use as input signals for one 
of the functions in the ``empix`` library.

In [ ]:
def generate_real_1d_input_signal():
    complex_1d_input_signal = generate_complex_1d_input_signal()
    
    real_1d_input_signal = complex_1d_input_signal.real
    real_1d_input_signal.metadata.General.title = "First Real 1D Input"
    real_1d_input_signal.metadata.y_intercept_of_imag_part = 0

    real_1d_input_signal.axes_manager[-1].name = "$r_{xy}$"
    real_1d_input_signal.axes_manager[-1].units = "Å"

    return real_1d_input_signal



def generate_complex_1d_input_signal():
    kwargs = {"data": generate_complex_1d_input_signal_data(), 
              "metadata": generate_complex_1d_input_signal_metadata()}
    complex_1d_input_signal = hyperspy.signals.ComplexSignal1D(**kwargs)

    axes = generate_complex_1d_input_signal_axes()

    for axis_idx, axis in enumerate(axes):
        complex_1d_input_signal.axes_manager[axis_idx].update_from(axis)
        complex_1d_input_signal.axes_manager[axis_idx].name = axis.name
        complex_1d_input_signal.axes_manager[axis_idx].units = axis.units

    return complex_1d_input_signal



def generate_complex_1d_input_signal_data():
    signal_data_shape = generate_complex_1d_input_signal_data_shape()
    Y_dim, X_dim, u_dim = signal_data_shape

    metadata = generate_complex_1d_input_signal_metadata()
    slopes = metadata["slopes"]
    y_intercept_of_real_part = metadata["y_intercept_of_real_part"]
    y_intercept_of_imag_part = metadata["y_intercept_of_imag_part"]

    kwargs = {"shape": signal_data_shape, "dtype": "complex"}
    signal_data = np.zeros(**kwargs)

    navigation_dims = (Y_dim, X_dim)
    num_patterns = Y_dim*X_dim

    axes = generate_complex_1d_input_signal_axes()
    d_u = axes[-1].scale

    n_u = np.arange(u_dim)

    for pattern_idx in range(num_patterns):
        navigation_indices = np.unravel_index(pattern_idx, navigation_dims)
        Y_idx, X_idx = navigation_indices

        u = n_u*d_u
        
        slope = slopes[Y_idx][X_idx]

        signal_data.real[navigation_indices] = (slope*u
                                                + y_intercept_of_real_part)
        signal_data.imag[navigation_indices] = (slope*u
                                                + y_intercept_of_imag_part)

    complex_1d_input_signal_data = signal_data

    return complex_1d_input_signal_data



def generate_complex_1d_input_signal_data_shape():
    complex_1d_input_signal_data_shape = (3, 2, 200)
    
    return complex_1d_input_signal_data_shape



def generate_complex_1d_input_signal_metadata():
    metadata = {"General": {"title": "First Complex 1D Input"}, 
                "Signal": dict(), 
                "slopes": generate_slopes(), 
                "y_intercept_of_real_part": 0,
                "y_intercept_of_imag_part": 10}

    complex_1d_input_signal_metadata = metadata

    return complex_1d_input_signal_metadata



def generate_slopes():
    signal_data_shape = generate_complex_1d_input_signal_data_shape()
    Y_dim, X_dim, r_dim = signal_data_shape
    
    slopes = np.zeros((Y_dim, X_dim))

    for Y_idx in range(Y_dim):
        for X_idx in range(X_dim):
            slopes[Y_idx, X_idx] = Y_idx*X_dim + X_idx

    slopes = slopes.tolist()

    return slopes



def generate_complex_1d_input_signal_axes():
    signal_data_shape = generate_complex_1d_input_signal_data_shape()
    Y_dim, X_dim, u_dim = signal_data_shape

    axes_sizes = (X_dim, Y_dim, u_dim)
    axes_scales = (1, 1, 0.1)
    axes_offsets = (0, 0, 0)
    axes_names = ("$X$", "$Y$", "$k_{xy}$")
    axes_units = ("mm", "mm", "1/Å")

    axes = tuple()
    for axis_idx, _ in enumerate(axes_names):
        kwargs = {"size": axes_sizes[axis_idx],
                  "scale": axes_scales[axis_idx],
                  "offset": axes_offsets[axis_idx],
                  "name": axes_names[axis_idx], 
                  "units": axes_units[axis_idx]}
        axis = hyperspy.axes.UniformDataAxis(**kwargs)
        axes += (axis,)

    complex_1d_input_signal_axes = axes

    return complex_1d_input_signal_axes



real_1d_input_signal = generate_real_1d_input_signal()
complex_1d_input_signal = generate_complex_1d_input_signal()

Let's plot the 1D input signals that we just created. Starting with the 
real-valued signal.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
real_1d_input_signal.plot(**kwargs)

And now the complex-valued signal.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
complex_1d_input_signal.plot(**kwargs)

### Calculating the modulus squared ###

Let's calculate the modulus squared of each input signal.

In [ ]:
kwargs = {"input_signal": real_2d_input_signal, "title": "foobar"}
output_signal_1 = empix.abs_sq(**kwargs)

kwargs = {"input_signal": complex_2d_input_signal, "title": None}
output_signal_2 = empix.abs_sq(**kwargs)

kwargs = {"input_signal": real_1d_input_signal, "title": None}
output_signal_3 = empix.abs_sq(**kwargs)

kwargs = {"input_signal": complex_1d_input_signal, "title": None}
output_signal_4 = empix.abs_sq(**kwargs)

Let's plot the resulting output signals.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_2.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_3.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_4.plot(**kwargs)

Note that for the first output signal, i.e. ``output_signal_1``, we overwrote
the automatically generated default title: rather than the title being
"Modulus Square of Real 2D Input", we chose the title to be "foobar".

Moreover, note that for each input signal, the metadata stored therein is copied
over to the corresponding output signal, with the title generally being modified
afterwards.

In [ ]:
real_2d_input_signal.metadata

In [ ]:
output_signal_1.metadata

All ``empix`` functions will copy the metadata from the input signal into the
corresponding output signal and then update the title.

### Azimuthally averaging ###

Let's azimuthally average each 2D input signal using the same optional averaging
parameters.

In [ ]:
# ``X_idx`` and ``Y_idx`` are x- and y-indices in navigation space.
Y_idx = 0
X_idx = 1

metadata = real_2d_input_signal.metadata
ring_centers = metadata.ring_centers
inner_ring_radius = metadata.inner_ring_radius
outer_ring_radius = metadata.outer_ring_radius

# ``ring_center`` and ``radial_range`` are in units of signal space axes.
ring_center = ring_centers[Y_idx][X_idx]
radial_range = (inner_ring_radius/2, outer_ring_radius+1)

num_bins = 180

kwargs = {"center": ring_center,
          "radial_range": radial_range,
          "num_bins": num_bins,
          "title": None}  # Automatically generate default title.
optional_params = empix.OptionalAzimuthalAveragingParams(**kwargs)

kwargs = {"input_signal": real_2d_input_signal, 
          "optional_params": optional_params}
output_signal_1 = empix.azimuthally_average(**kwargs)

kwargs = {"input_signal": complex_2d_input_signal, 
          "optional_params": optional_params}
output_signal_2 = empix.azimuthally_average(**kwargs)

Let's plot the resulting output signals.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_2.plot(**kwargs)

### Azimuthally integrating ###

Let's azimuthally integrate each 2D input signal using the same optional
integration parameters.

In [ ]:
kwargs = {"center": ring_center,
          "radial_range": radial_range,
          "num_bins": num_bins,
          "title": None}
optional_params = empix.OptionalAzimuthalIntegrationParams(**kwargs)

kwargs = {"input_signal": real_2d_input_signal, 
          "optional_params": optional_params}
output_signal_1 = empix.azimuthally_integrate(**kwargs)

kwargs = {"input_signal": complex_2d_input_signal, 
          "optional_params": optional_params}
output_signal_2 = empix.azimuthally_integrate(**kwargs)

Let's plot the resulting output signals.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_2.plot(**kwargs)

### Annularly averaging ###

Let's annularly average each 2D input signal using the same optional averaging
parameters.

In [ ]:
kwargs = {"center": ring_center,
          "radial_range": radial_range,
          "title": None}
optional_params = empix.OptionalAnnularAveragingParams(**kwargs)

kwargs = {"input_signal": real_2d_input_signal, 
          "optional_params": optional_params}
output_signal_1 = empix.annularly_average(**kwargs)

kwargs = {"input_signal": complex_2d_input_signal, 
          "optional_params": optional_params}
output_signal_2 = empix.annularly_average(**kwargs)

Let's plot the resulting output signals.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_2.plot(**kwargs)

### Annularly integrating ###

Let's annularly average each 2D input signal using the same optional integration
parameters.

In [ ]:
kwargs = {"center": ring_center,
          "radial_range": radial_range,
          "title": None}
optional_params = empix.OptionalAnnularIntegrationParams(**kwargs)

kwargs = {"input_signal": real_2d_input_signal, 
          "optional_params": optional_params}
output_signal_1 = empix.annularly_integrate(**kwargs)

kwargs = {"input_signal": complex_2d_input_signal, 
          "optional_params": optional_params}
output_signal_2 = empix.annularly_integrate(**kwargs)

Let's plot the resulting output signals.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_2.plot(**kwargs)

### Cropping ###

Let's crop each 2D input signal using the same optional cropping parameters.

In [ ]:
kwargs = {"center": ring_center,
          "window_dims": (190, 185),
          "pad_mode": "zeros",
          "apply_symmetric_mask": True,
          "title": None}
optional_params = empix.OptionalCroppingParams(**kwargs)

kwargs = {"input_signal": real_2d_input_signal, 
          "optional_params": optional_params}
output_signal_1 = empix.crop(**kwargs)

kwargs = {"input_signal": complex_2d_input_signal, 
          "optional_params": optional_params}
output_signal_2 = empix.crop(**kwargs)

Let's plot the resulting output signals.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_2.plot(**kwargs)

### Downsampling ###

Let's downsample each 2D input signal using the same optional cropping
parameters.

In [ ]:
kwargs = {"block_dims": (2, 3),
          "padding_const": 0,
          "downsample_mode": "sum",
          "title": None}
optional_params = empix.OptionalDownsamplingParams(**kwargs)

kwargs = {"input_signal": real_2d_input_signal, 
          "optional_params": optional_params}
output_signal_1 = empix.downsample(**kwargs)

kwargs = {"input_signal": complex_2d_input_signal, 
          "optional_params": optional_params}
output_signal_2 = empix.downsample(**kwargs)

Let's plot the resulting output signals.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_2.plot(**kwargs)

### Resampling ###

Let's resample each 2D input signal using the same optional resampling
parameters.

In [ ]:
h_size = real_2d_input_signal.axes_manager[-2].size
h_scale = real_2d_input_signal.axes_manager[-2].scale
h_offset = real_2d_input_signal.axes_manager[-2].offset

v_size = real_2d_input_signal.axes_manager[-1].size
v_scale = real_2d_input_signal.axes_manager[-1].scale
v_offset = real_2d_input_signal.axes_manager[-1].offset

kwargs = {"new_signal_space_sizes": (2*h_size, 2*v_size),
          "new_signal_space_scales": (h_scale/2, v_scale/2),
          "new_signal_space_offsets": (h_offset+h_scale/2, v_offset+v_scale/2),
          "spline_degrees": (3, 3),
          "interpolate_polar_cmpnts": True,
          "title": None}
optional_params = empix.OptionalResamplingParams(**kwargs)

kwargs = {"input_signal": real_2d_input_signal, 
          "optional_params": optional_params}
output_signal_1 = empix.resample(**kwargs)

kwargs = {"input_signal": complex_2d_input_signal, 
          "optional_params": optional_params}
output_signal_2 = empix.resample(**kwargs)

Let's plot the resulting output signals.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_2.plot(**kwargs)

### Cumulatively integrating 1D signals ###

Let's cumulatively integrate the real 1D input signal.

In [ ]:
u_size = real_1d_input_signal.axes_manager[-1].size
u_scale = real_1d_input_signal.axes_manager[-1].scale
u_offset = real_1d_input_signal.axes_manager[-1].offset

kwargs = {"limits": (u_offset+20*u_scale, u_offset+130*u_scale),
          "num_bins": num_bins,
          "normalize": True,
          "title": None}
optional_params = empix.OptionalCumulative1dIntegrationParams(**kwargs)

kwargs = {"input_signal": real_1d_input_signal, 
          "optional_params": optional_params}
output_signal_1 = empix.cumulatively_integrate_1d(**kwargs)

Let's plot the resulting output signal.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 1,
          "cmap": "jet"}
output_signal_1.plot(**kwargs)